In [1]:
import os
import h5py
import numpy as np
from tqdm import tqdm

# -----------------------
# 用户配置
# -----------------------

file_path = "h5file/TSPZ155B0024_TSPZ155B0025_-800V_BLR-high_Static-off_Non-para-off_R400us-acc-0_94cm_30C_42%_X001-CT_NONE_2025-08-22_raw.h5"
# file_path = "TSPZ158B0255_TSPZ158B0256_lab535.h5"
# 取文件名（去掉路径和扩展名）
fname = os.path.splitext(os.path.basename(file_path))[0]
# parts = fname.split("_")
parts = fname.split("_")
print(parts)
leftcrystal = parts[0]
rightcrystal = parts[1]
lab = "_".join(parts[3:])
print(lab)

# 指定输出文件夹
base_output_dir = "output_raw"
os.makedirs(base_output_dir, exist_ok=True)
# 左右晶体各自的输出目录
left_output_dir = os.path.join(base_output_dir, leftcrystal)
right_output_dir = os.path.join(base_output_dir, rightcrystal)
os.makedirs(left_output_dir, exist_ok=True)
os.makedirs(right_output_dir, exist_ok=True)

# 内置一个测试条件列表，只保留含有这些关键词的 Averaged Data
conditions = [
    # "100kV-10mA-BLR-Nonpara-off-250us",
    # "100kv-10mA-BLR-Nonpara-off-400us",
    # "100kv-BLR-Nonpara-on-33.33MHz-400us",
    # "100kv-BLR-Nonpara-on-40MHz-400us",
    # "100kv-BLR-Nonpara-on-66.67MHz-400us",
    # "100kv-BLR-Nonpara-on-80MHz-400us",

    # "120kv-10mA-BLR-Nonpara-off-250us",
    # "120kv-10mA-BLR-Nonpara-off-400us",
    # "120kv-10mA-BLR-Nonpara-on-30.77MHz-400us",
    # "120kv-10mA-BLR-Nonpara-on-33.33MHz-400us",
    # "120kv-10mA-BLR-Nonpara-on-40MHz-400us",
    # "120kv-10mA-BLR-Nonpara-on-66.67MHz-400us",
    # "120kv-10mA-BLR-nONPARA-ON-80mhZ-400us",

    # "120kv-10mA-Staticleakage-Nonpara-off-40MHz-400us",
    # "120kv-10mA-noleakagecomp-Nonpara-off-400us",
    # "120kv-10mA-noleakagecomp-Nonpara-on-40MHz-400us",
    # "120kv-10mA-Staticleakage-Nonpara-on-40MHz-400us",

    # "dark-BLR-Nonpara-off-400us",
    # "dark-BLR-Nonpara-on-33.33MHz-400us",
    # "dark-BLR-Nonpara-on-40MHz-400us",
    # "dark-BLR-Nonpara-on-66.67MHz-400us",
    # "dark-BLR-Nonpara-on-80MHz-400us",
    
    # "dark-noleakagecomp-Nonpara-off-400us",
    # "dark-noleakagecomp-Nonpara-on-40MHz-400us",
    # "dark_Staticleakage_Nonpara-off_400us",
    # "dark_noleakage_Nonpara-off_400us",

    ## 文件 TSPZ155B0024+TSPZ155B0025_-800V_BLR-high_Static-off_Non-para-off_R400us-acc-0_94cm_30C_42%_X001-CT_NONE_2025-08-22_raw 内的格式
    "100Kv-40mA",
    "120kV-40mA",
    "dark"
]

nrow, ncol = 24, 36 

# -----------------------
# 读取数据集
# -----------------------
with h5py.File(file_path, "r") as f:
    raw_group = f["Raw Data"]

    # 找到所有 Averaged Data 的 key
    dataset_keys = [k for k in raw_group.keys() if k.endswith("Averaged Data")]

    # 过滤：只保留用户设定的条件
    selected_keys = []
    for cond in conditions:
        for k in dataset_keys:
            if cond.lower() in k.lower():   # 全部转小写再匹配
                selected_keys.append(k)

    print("selected_keys 的数量 :", len(selected_keys))
    print("匹配到的数据集：", selected_keys)

    # -----------------------
    # 数据处理
    # -----------------------
    for key in tqdm(selected_keys, desc="Processing conditions"):
        data = raw_group[key][:]   # (255, 5, 1727)

        # 5 通道平均
        avg_data = np.mean(data, axis=1)   # (255, 1727)

        # 拆分左右晶体
        left = avg_data[:, :864].reshape(-1, nrow, ncol).astype(np.uint32)   # (255, 864)
        right = avg_data[:, 864:].reshape(-1, nrow, ncol).astype(np.uint32)  # (255, 863)
 
        # 保存为 .raw 文件 
        clean_key = key.replace("Threshold Scan Averaged Data", "").strip()
        left.tofile(os.path.join(left_output_dir, f"{leftcrystal}_{clean_key}_{lab}.raw"))
        right.tofile(os.path.join(right_output_dir,f"{rightcrystal}_{clean_key}_{lab}.raw"))
    
        print(f"已写出: {leftcrystal}_{key}_{lab}.raw, {leftcrystal}_{key}_{lab}.raw")


['TSPZ155B0024', 'TSPZ155B0025', '-800V', 'BLR-high', 'Static-off', 'Non-para-off', 'R400us-acc-0', '94cm', '30C', '42%', 'X001-CT', 'NONE', '2025-08-22', 'raw']
BLR-high_Static-off_Non-para-off_R400us-acc-0_94cm_30C_42%_X001-CT_NONE_2025-08-22_raw
selected_keys 的数量 : 3
匹配到的数据集： ['100kV-40mA Threshold Scan Averaged Data', '120kV-40mA Threshold Scan Averaged Data', 'dark Threshold Scan Averaged Data']


Processing conditions: 100%|██████████| 3/3 [00:00<00:00, 86.30it/s]

已写出: TSPZ155B0024_100kV-40mA Threshold Scan Averaged Data_BLR-high_Static-off_Non-para-off_R400us-acc-0_94cm_30C_42%_X001-CT_NONE_2025-08-22_raw.raw, TSPZ155B0024_100kV-40mA Threshold Scan Averaged Data_BLR-high_Static-off_Non-para-off_R400us-acc-0_94cm_30C_42%_X001-CT_NONE_2025-08-22_raw.raw
已写出: TSPZ155B0024_120kV-40mA Threshold Scan Averaged Data_BLR-high_Static-off_Non-para-off_R400us-acc-0_94cm_30C_42%_X001-CT_NONE_2025-08-22_raw.raw, TSPZ155B0024_120kV-40mA Threshold Scan Averaged Data_BLR-high_Static-off_Non-para-off_R400us-acc-0_94cm_30C_42%_X001-CT_NONE_2025-08-22_raw.raw
已写出: TSPZ155B0024_dark Threshold Scan Averaged Data_BLR-high_Static-off_Non-para-off_R400us-acc-0_94cm_30C_42%_X001-CT_NONE_2025-08-22_raw.raw, TSPZ155B0024_dark Threshold Scan Averaged Data_BLR-high_Static-off_Non-para-off_R400us-acc-0_94cm_30C_42%_X001-CT_NONE_2025-08-22_raw.raw
